In [1]:
%load_ext autoreload
%autoreload 2

# Score climatology

Compute and score the climatology, to use as a baseline.

In [2]:
import dask
import dask.array as da
import dask.distributed
import dask_jobqueue
import datetime
import pathlib
import xarray as xr

In [3]:
from crims2s.util import fix_dataset_dims

{'array': {'slicing': {'split_large_chunks': True}}}


In [ ]:
!ls ~/.config/dask

In [4]:
dask.config.get('array.slicing')

{'split-large-chunks': True}

In [ ]:
INPUT_TRAIN = '***BASEDIR***training-input/0.3.0/netcdf'
OBSERVATIONS = '***BASEDIR***training-output-reference/'

## Boost dask cluster

In [ ]:
cluster = dask_jobqueue.SLURMCluster(
    env_extra=['source ***HOME***.bash_profile','conda activate s2s'],
)

In [ ]:
cluster.scale(jobs=3)  # Scale to two working nodes as configured.
client = dask.distributed.Client(cluster)

In [ ]:
client

# Open model data

In [ ]:
CENTER = 'ecmwf'
FIELD = 't2m'

In [ ]:
input_path = pathlib.Path(INPUT_TRAIN)

In [ ]:
files = sorted([f for f in input_path.iterdir() if CENTER in f.stem and FIELD in f.stem])

In [ ]:
files[:10]

In [ ]:
dataset = xr.open_mfdataset(files, preprocess=fix_dataset_dims)

In [ ]:
dataset

In [ ]:
valid_time = dataset.valid_time.compute()

In [ ]:
valid_time[0, 0]

## Open observations

In [ ]:
obs_path = pathlib.Path(OBSERVATIONS)
obs_files = [f for f in obs_path.iterdir() if 't2m' in f.stem]

In [ ]:
d = xr.open_dataset(obs_files[0])

In [ ]:
fix_dataset_dims(d)

In [ ]:
obs = xr.open_mfdataset(obs_files, preprocess=fix_dataset_dims)

In [ ]:
obs

In [ ]:
obs_weeks34 = obs.isel(lead_time=range(14, 28))

In [ ]:
dask.config.get('array')